## 1. 라이브러리 및 데이터 불러오기

In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings('ignore')

In [97]:
jobplanet = pd.read_excel('C:/Users/hanah/Desktop/workspace/Project/ESG/crawling/jobplanet/jobplanet_review.xlsx')

## 2. 작성일자 기준으로 해당년도 구분

In [98]:
jobplanet['no'] = 0
cmp_dict = {}
for i in range(len(jobplanet)):
    cmp_dict[jobplanet.회사명[i]] = cmp_dict.get(jobplanet.회사명[i], 0) + 1 
    jobplanet.loc[i, 'no'] = cmp_dict[jobplanet.회사명[i]]

jobplanet.set_index(['회사명', 'no'], inplace=True)

In [99]:
jobplanet.index

MultiIndex([('블리츠웨이', 1),
            ('블리츠웨이', 2),
            ('블리츠웨이', 3),
            ('블리츠웨이', 4),
            ('석경에이티', 1),
            ('석경에이티', 2),
            ('석경에이티', 3),
            ('석경에이티', 4),
            ('석경에이티', 5),
            ('석경에이티', 6),
            ...
            ('엠벤처투자', 3),
            ('엠벤처투자', 4),
            ('엠벤처투자', 5),
            (  '모헨즈', 1),
            (  '모헨즈', 2),
            (  '모헨즈', 3),
            (  '모헨즈', 4),
            (  '모헨즈', 5),
            (  '모헨즈', 6),
            (  '모헨즈', 7)],
           names=['회사명', 'no'], length=58793)

In [100]:
jobplanet['작성년도'] = 0
for i in range(len(jobplanet)):
    if jobplanet.index[i][1] <= cmp_dict[(jobplanet.index[i][0])]//2:
        jobplanet['작성년도'][i] = 2022

C:\Users\hanah\AppData\Local\Temp\ipykernel_12056\1182551494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobplanet['작성년도'][i] = 2022


In [111]:
jobplanet['작성년도'].value_counts()

2020    29608
2021    29185
Name: 작성년도, dtype: int64

In [114]:
jobplanet.reset_index(inplace=True)
jobplanet.drop(['no', '작성일'], axis=1, inplace=True)

## 3. 리뷰별 평균평점 구하기

In [4]:
temp = jobplanet.iloc[:, :6]
temp.replace('%','', regex=True, inplace=True)
temp.iloc[:, 1:6] = temp.iloc[:, 1:6].astype(int)

In [5]:
jobplanet = jobplanet.iloc[:, 6:]
jobplanet = pd.concat([temp, jobplanet], axis=1)
jobplanet['평균평점'] = temp.mean(axis=1)
# temp.groupby('회사명').mean().reset_index().mean(axis=1)
jobplanet

,회사명,승진 기회 및 가능성,복지 및 급여,업무와 삶의 균형,사내문화,경영진,총평,리뷰,작성년도,평균평점
0,블리츠웨이,40,80,40,40,60,"""능력있는 사람에게는 기회나 보상이 주어지는 회사""","장점다양한 경험 가능, 실무의 A부터 Z까지 배울수 있음, 서울에 회사 사옥이 있...",2021,52.0
1,블리츠웨이,20,20,20,80,20,"""성장하는 회사 그만큼 직원들을 굴리며 갈아버리는 회사.""","장점팀 분위기가 좋다, 쾌적한 시설과 환경, 편리한 교통과 좋은 위치.단점워라밸 포...",2021,32.0
2,블리츠웨이,20,20,20,40,20,"""원수가 다니겠다고 하면 도시락 싸 들고 환영할 회사._x000D_중소기업의 흔한 ...",장점* 꼰대 문화가 별로 없고 변화와 발전 추구함. 투자를 많이 받고 있기 때문에 ...,2020,24.0
3,블리츠웨이,60,60,40,100,100,"""앞으로도 더 성장 할 가능성이 높은 회사""",장점직원들에게 많은 기회를 준다. 한가지만 보지 않고 다양한 분야에서 어떻게 인재가...,2020,72.0
4,석경에이티,20,20,40,20,20,"""업무에 있어서 배울점은 많은곳 하지만 배운게 많아도 대우는 못받는곳""",장점5일만 출근한다는개 장점 그외에는 어느 회사를 가든 다 있을 법한 텃새 혹은 꼰...,2021,24.0
...,...,...,...,...,...,...,...,...,...,...
58588,모헨즈,60,40,80,40,40,"""안정적이고 급여도 괜찮지만 의사결정이 느리고 경직된 조직문화""",장점연차를 눈치 안보고 쓸수 있음. 업무가 단조로워 접근성이 쉬움. 야근이 없음. ...,2021,52.0
58589,모헨즈,100,80,40,40,80,"""넉넉한 급여와 충분한 복지를 누릴 수 있는곳, 다만 사내정치만 조금 조심하시길""","장점남자로서 정말 열심히 일하면 적당한 보상이 따라오는곳, 레미콘회사치고 높은급여 ...",2020,68.0
58590,모헨즈,40,80,80,20,20,"""오래된 사고방식에서 벗어나지 못하는 보수주의 회사의 결정체""",장점업무량대비 꽤 높은 임금 & 꾸준한 임금 인상(재직당시 기준)단점비합리적인 인사...,2020,48.0
58591,모헨즈,40,40,20,20,20,"""레미콘제조업체로 현상유지만 하고있음.앞으로 전망은 낙관적이지 않음.""",장점공장은 장점이 없고 서울본사는 근무할만 함.빽있는 사람은 다닐만함.단점빽없는 사...,2020,28.0


In [6]:
jobplanet.to_excel('jobplanet.xlsx', index=False)